In [37]:
# importing the relevant packages
import pandas as pd
import numpy as np
from drf import drf

# generate data
n = 1000
p = 10
d = 2
X = np.random.normal(0, 1, size=(n, p))
Y = np.random.normal(0, 1, size=(n, d))
Y[:,0] = Y[:,0] + X[:,0] #mean shift of Y1 based on X1
Y[:,1] = Y[:,1] * X[:,1] #variance shift of Y2 based on X2
X = pd.DataFrame(X)
Y = pd.DataFrame(Y)

# fit model
DRF = drf(min_node_size = 15, num_trees = 2000, splitting_rule = "FourierMMD", compute_variable_importance = True) #those are the default values
DRF.fit(X, Y)
DRF.info() #prints variable importance


DRF forest object
Number of trees: 2000 
Number of training samples: 1000 
Variable importance: 
          0           1           2           3           4           5 
0.129554819 0.067101154 0.004398879 0.004110650 0.003529065 0.002928701 
          6           7           8           9 
0.003224854 0.002654478 0.003544571 0.003235930 


In [35]:
DRF.variable_importance()

<rpy2.rinterface_lib.sexp.NULLType object at 0x7f94907d17c8> [RTYPES.NILSXP]

In [29]:
#generate test data
X_test = pd.DataFrame(np.random.normal(0, 1, size=(100, p)))

# estimated conditional distribution represented via weights
out = DRF.predict(newdata = X_test)
print(out.weights)
# many distributional functionals are implemented and do not need to be manually computed from the weights  
out = DRF.predict(newdata = X_test, functional = "mean")
print(out.mean)
print(out.mean.shape)
# covariance matrix at a fixed test point
out = DRF.predict(newdata = [0]*p, functional = "cov")
print(out.cov[0,:,:])
print(out.cov.shape)
# we can transform the response beforehand to obtain more complicated quantities 
out = DRF.predict(newdata = X_test, functional = "quantile", transformation = lambda y: (np.sin(y[1]), y[1]*y[2], y[2]**2), quantiles=[0.1, 0.9])
print(out.quantile[0,1,:]) # 0.1 and 0.9 quantiles for first test point in newdata and for the second component of transformed y
print(out.quantile.shape)


R[write to console]: Error in (function (object, newdata = NULL, transformation = NULL, functional = NULL,  : 
  newdata should have the same dimension as the training data.



RRuntimeError: Error in (function (object, newdata = NULL, transformation = NULL, functional = NULL,  : 
  newdata should have the same dimension as the training data.


In [22]:
# we automatically handle factor variables by using one-hot encoding
X['cat'] = np.random.choice(['a', 'b', 'c', 'd', 'e'], n, replace=True)
Y['new'] = np.random.normal(0, 1, size=n) + (X['cat']=='a')
DRF.fit(X, Y)
DRF.info()

DRF forest object
Number of trees: 2000 
Number of training samples: 1000 


(100, 10)